# Autonomous driving - Car detection

# 1 - Statement
目标检测

- $p_c$ 是不是这个目标
- $b_x$ $b_y$ 目标中心
- $b_h$ $b_w$ 目标长宽
- $c$ 属于哪一类?


<img src="https://wx1.sinaimg.cn/mw1024/701c57e5gy1gen5j500lcj20u60gu127.jpg" style="width:600px;">

# 2. YOLO

#### Inputs / Outputs
* Model Inputs: (m, 608, 608, 3)
* Outputs: (pc, bx, by, bh, bw, c)
* 这里我们有80个分类 (pc, bx, by, bh, bw, c1, ....., c80) 共85个返回值
* CNN: 缩放因子是32, 所以(608, 608, 3) => 缩放后变成608/32 = 19 => 19 * 19
* (m, 608, 608, 3) => CNN (m, 19, 19)

**简单理解为 图像经过CNN 压缩处理后, 将图像分为 19 * 19块儿, 每块儿都是 32 * 32 * 3 的压缩结果**

#### Anchor Boxes
* 解决多个对象在同一个框里的问题
* 因为我需要对19 * 19 每一个块儿进行检查, 一块儿里面, 有5个Anchor Boxes和对应的分类检测结果
* 这里我们 有5个Anchor Boxes 来覆盖80个种类classes
* $(m, n_H, n_W, anchors, classes)$: $(m, 19, 19, 5, 85)$

#### Process
* image => CNN => ENCODING

#### Encoding
* Encoding去找图像边界
* $(m, n_H, n_W, anchors, classes)$
* 经过CNN图像处理后的尺寸是(m, 19, 19, 5, 85)
* m个样本, 一个样本尺寸为 (19, 19, 5, 85)
* $n_H, n_W$ 疑问: CNN后图像的尺寸吧?
* 5: 有5个 Anchor Boxes
* 85: 5个定位类型, 80个分类判断
![image](https://wx1.sinaimg.cn/mw1024/701c57e5gy1gencqznpy9j21d80ti7kz.jpg)

#### Flatten
![image](https://wx1.sinaimg.cn/mw1024/701c57e5gy1gendhka776j20ye0u0k36.jpg)


#### Class Score
* 将图像19 * 19 其中的一个块儿, 相当于一个小图像拿出来
* 里面有5个检测框的目标检测, 一个检测框分别对应有80分类+5个(pc, bx, by, bh, bw, c)
* 一个检测框 box1, pc为1, 则去找对class分类概率谁最高
![image](https://wx1.sinaimg.cn/mw1024/701c57e5gy1gendhjw4s7j215k0rwgtj.jpg)

#### Picture
![image](https://wx3.sinaimg.cn/mw1024/701c57e5gy1genfjajm6jj215w0ryb29.jpg)

In [5]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline